# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~


len = 17
cluster0
hz
lru_clock
used_memory_peak
used_memory_overhead
allocator_allocated
allocator_active
maxmemory
maxmemory_human
allocator_frag_bytes
rss_overhead_ratio
rss_overhead_bytes
rdb_last_save_time
instantaneous_ops_per_sec
instantaneous_input_kbps
instantaneous_output_kbps
used_cpu_sys
used_cpu_user

len = 7
cluster1
used_memory_rss
used_memory_rss_human
allocator_resident
allocator_rss_ratio
allocator_rss_bytes
mem_fragmentation_ratio
mem_fragmentation_bytes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
# internal metrics 분류 결과
cluster0 =[ 
'hz',
'lru_clock',
'used_memory_peak',
'used_memory_overhead',
'allocator_allocated',
'allocator_active',
'maxmemory',
'maxmemory_human',
'allocator_frag_bytes',
'rss_overhead_ratio',
'rss_overhead_bytes',
'rdb_last_save_time',
'instantaneous_ops_per_sec',
'instantaneous_input_kbps',
'instantaneous_output_kbps',
'used_cpu_sys',
'used_cpu_user',
]

cluster1 = [
'used_memory_rss',
'used_memory_rss_human',
'allocator_resident',
'allocator_rss_ratio',
'allocator_rss_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes',
]

knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [10]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics_r10000_rdb.csv'

internal_csv = pd.read_csv(file_dir+file_name)

# 1250 개 추출
internal_csv = internal_csv.iloc[0:1250,:]
internal_csv.shape

(1250, 124)

In [11]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config_r10000_rdb.csv'

knobs = pd.read_csv(file_dir + file_name)
# 1250 개 추출
knobs = knobs.iloc[0:1250,:]
knobs.shape

(1250, 32)

In [12]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [13]:
# 클러스터 합치기
cluster = cluster0+cluster1

In [14]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

In [15]:
# throughput 결측치 제거
total_internal_df.dropna()

,hz,lru_clock,used_memory_peak,used_memory_overhead,allocator_allocated,allocator_active,maxmemory,maxmemory_human,allocator_frag_bytes,rss_overhead_ratio,...,instantaneous_output_kbps,used_cpu_sys,used_cpu_user,used_memory_rss,used_memory_rss_human,allocator_resident,allocator_rss_ratio,allocator_rss_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes
0,23,4253746,1241152,797176,1192120,1445888,2097152000,1.95,253768,0.92,...,70.79,0.199604,0.092045,4079616,3.89,4415488,3.05,2969600,3.52,2920496
1,36,4253766,1241096,797144,1196384,1445888,1258291200,1.17,249504,0.91,...,75.16,0.231897,0.107341,4055040,3.87,4456448,3.08,3010560,3.50,2895952
2,37,4253778,1241096,797144,1196192,1445888,1887436800,1.76,249696,0.90,...,75.17,0.206786,0.107529,4075520,3.89,4546560,3.14,3100672,3.52,2916432
3,16,4253790,1241096,797144,1196368,1445888,2621440000,2.44,249520,0.91,...,98.20,0.225227,0.101039,4063232,3.88,4472832,3.09,3026944,3.51,2904144
4,21,4253802,1241096,797144,1196368,1445888,2202009600,2.05,249520,0.91,...,64.93,0.216427,0.099726,4059136,3.87,4456448,3.08,3010560,3.50,2900048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,8,4269043,1241096,797144,1196392,1445888,2621440000,2.44,249496,0.91,...,48.70,0.186578,0.104191,4059136,3.87,4472832,3.09,3026944,3.50,2900048
1246,37,4269055,1241096,797144,1196192,1445888,1258291200,1.17,249696,0.92,...,75.30,0.204926,0.088129,4059136,3.87,4411392,3.05,2965504,3.50,2900048
1247,10,4269067,1241096,797144,1196200,1445888,1992294400,1.86,249688,0.92,...,60.95,0.205735,0.084164,4055040,3.87,4411392,3.05,2965504,3.50,2895952
1248,38,4269079,1241096,797144,1196768,1445888,1468006400,1.37,249120,0.91,...,78.14,0.210007,0.096677,4067328,3.88,4472832,3.09,3026944,3.64,2950168


In [16]:
# internal_csv 결측치 확인 -> 없음!
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   hz                         1250 non-null   int64  
 1   lru_clock                  1250 non-null   int64  
 2   used_memory_peak           1250 non-null   int64  
 3   used_memory_overhead       1250 non-null   int64  
 4   allocator_allocated        1250 non-null   int64  
 5   allocator_active           1250 non-null   int64  
 6   maxmemory                  1250 non-null   int64  
 7   maxmemory_human            1250 non-null   float64
 8   allocator_frag_bytes       1250 non-null   int64  
 9   rss_overhead_ratio         1250 non-null   float64
 10  rss_overhead_bytes         1250 non-null   int64  
 11  rdb_last_save_time         1250 non-null   int64  
 12  instantaneous_ops_per_sec  1250 non-null   int64  
 13  instantaneous_input_kbps   1250 non-null   float

In [17]:
# maxmemory 처리 throughtput은 처리할 필요없다.
total_internal_df['maxmemory']

0       2097152000
1       1258291200
2       1887436800
3       2621440000
4       2202009600
           ...    
1245    2621440000
1246    1258291200
1247    1992294400
1248    1468006400
1249    1572864000
Name: maxmemory, Length: 1250, dtype: int64

In [18]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [19]:
# knob_list 만 추출, AOF 관련 제외
subTotal_knobs = extract_data(knob_list, knobs)

In [20]:
subTotal_knobs

,save0_0,save0_1,save1_0,save1_1,save2_0,save2_1,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,1056,4,114,13,57,8497,yes,no,no,NaN,...,volatile-lru,6,yes,no,no,615,232,no,23,no
1,805,2,343,23,47,8561,no,yes,no,NaN,...,allkeys-lru,7,yes,no,no,496,156,yes,36,no
2,776,6,194,46,61,11337,no,yes,yes,yes,...,volatile-random,5,no,yes,no,494,135,no,37,no
3,742,9,295,59,57,8989,yes,yes,yes,NaN,...,allkeys-lru,7,no,no,no,491,76,yes,16,no
4,985,5,225,90,41,11978,no,no,no,NaN,...,volatile-lru,5,yes,no,yes,332,80,yes,21,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,851,5,121,84,79,11338,yes,no,no,NaN,...,noeviction,3,no,yes,no,623,244,no,8,no
1246,825,8,443,19,49,10608,yes,no,yes,yes,...,noeviction,7,no,yes,yes,363,34,no,37,no
1247,908,7,255,88,64,8293,no,no,yes,yes,...,volatile-ttl,7,no,no,yes,572,229,yes,10,no
1248,1089,5,179,81,40,8054,no,yes,yes,yes,...,allkeys-lfu,6,yes,yes,no,375,32,no,38,yes


In [21]:
#activedefrag knobs 분리

# activedefrag O 존재
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# activedefrag X 존재하지 않는 나머지 반
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X 전체 
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [22]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# 수치, 불리언, 불리언아닌것 (activedefrag)
numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [23]:
# activedefrag O
total_internal_Active = total_internal_df.iloc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.iloc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.iloc[total_index]

---

## 상관분석 함수

In [24]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [25]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


# knob하나 internal 여러개 상관계수 구하기 이상치 제거 버전
# pearson
## 모수 검정
def cal_pearsonr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pearsonr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = kendalltau(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

    
# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = spearmanr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])



In [26]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

def extract_pvalue_outlier(knobs, internal_df):
    print('pearson')
    cal_pearsonr_outlier(knobs, internal_df)
    print()
    
    print('kendalltau')
    cal_kendalltau_outlier(knobs, internal_df)
    print()
    
    print('spearman')
    cal_spearmanr_outlier(knobs, internal_df)
    print()
    

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)
        
        
# knob 리스트 각각 별로 extract_pvalue 수행 이상치 제거 
def extract_pvalue_knob_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue_outlier(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [27]:
numericalActive_var
numericalActive_var_new = numericalActive_var[6:10]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [28]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-threshold-upper###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson
                   corr   p-value
used_cpu_user  0.080446  0.045427

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-max###
pearson
                   corr   p-value
used_cpu_user -0.092388  0.021513

kendalltau
                          corr   p-value
allocator_frag_bytes  0.059657  0.049021
used_cpu_user        -0.075359  0.006141

spearman
                   corr   p-value
used_cpu_user -0.112259  0.005171



In [29]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

In [30]:
active_result

,cluster0,cluster1
active-defrag-threshold-lower,0.035180,0.030284
active-defrag-threshold-upper,0.027357,0.022808
active-defrag-cycle-min,0.022081,0.021410
active-defrag-cycle-max,0.033314,0.025537


In [31]:
# knob이 어떤 클러스터에 속하는지 확인 가능 함수
def get_cluster(df):
    cluster_list = df.columns.tolist()  # [cluster0, cluster1]
    for knob_index in list(df.index):
        max_index = np.argmax(df.loc[knob_index,:].values)
        print(knob_index+' : ', end='')
        print(cluster_list[max_index])    

In [32]:
get_cluster(active_result)

active-defrag-threshold-lower : cluster0
active-defrag-threshold-upper : cluster0
active-defrag-cycle-min : cluster0
active-defrag-cycle-max : cluster0


### 이상치 제거 후 상관계수

In [53]:
extract_pvalue_knob_outlier(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson

kendalltau

spearman

###active-defrag-threshold-upper###
pearson

kendalltau

spearman

###active-defrag-cycle-min###
pearson

kendalltau

spearman

###active-defrag-cycle-max###
pearson

kendalltau

spearman



---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [33]:
numerical_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [34]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson
                         corr   p-value
used_memory_peak     0.069579  0.013875
allocator_allocated -0.059752  0.034660
allocator_active    -0.058876  0.037405

kendalltau
                      corr  p-value
allocator_active -0.048094  0.03725

spearman
                      corr   p-value
allocator_active -0.058852  0.037484

###save0_1###
pearson
                           corr   p-value
hz                    -0.060676  0.031950
used_memory_rss        0.065124  0.021300
used_memory_rss_human  0.059347  0.035906

kendalltau
                             corr   p-value
hz                      -0.042579  0.034533
maxmemory_human         -0.040261  0.048328
used_memory_rss          0.046505  0.028543
mem_fragmentation_ratio  0.045915  0.036619
mem_fragmentation_bytes  0.047543  0.022474

spearman
                             corr   p-value
hz                      -0.059611  0.035089
maxmemory_human         -0.056976  0.044006
used_memory_rss          0.061633  0.0293

In [35]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,hz,lru_clock,used_memory_peak,used_memory_overhead,allocator_allocated,allocator_active,maxmemory,maxmemory_human,allocator_frag_bytes,rss_overhead_ratio,...,instantaneous_output_kbps,used_cpu_sys,used_cpu_user,used_memory_rss,used_memory_rss_human,allocator_resident,allocator_rss_ratio,allocator_rss_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes
0,23,4253746,1241152,797176,1192120,1445888,2097152000,1.95,253768,0.92,...,70.79,0.199604,0.092045,4079616,3.89,4415488,3.05,2969600,3.52,2920496
1,36,4253766,1241096,797144,1196384,1445888,1258291200,1.17,249504,0.91,...,75.16,0.231897,0.107341,4055040,3.87,4456448,3.08,3010560,3.50,2895952
2,37,4253778,1241096,797144,1196192,1445888,1887436800,1.76,249696,0.90,...,75.17,0.206786,0.107529,4075520,3.89,4546560,3.14,3100672,3.52,2916432
3,16,4253790,1241096,797144,1196368,1445888,2621440000,2.44,249520,0.91,...,98.20,0.225227,0.101039,4063232,3.88,4472832,3.09,3026944,3.51,2904144
4,21,4253802,1241096,797144,1196368,1445888,2202009600,2.05,249520,0.91,...,64.93,0.216427,0.099726,4059136,3.87,4456448,3.08,3010560,3.50,2900048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,8,4269043,1241096,797144,1196392,1445888,2621440000,2.44,249496,0.91,...,48.70,0.186578,0.104191,4059136,3.87,4472832,3.09,3026944,3.50,2900048
1246,37,4269055,1241096,797144,1196192,1445888,1258291200,1.17,249696,0.92,...,75.30,0.204926,0.088129,4059136,3.87,4411392,3.05,2965504,3.50,2900048
1247,10,4269067,1241096,797144,1196200,1445888,1992294400,1.86,249688,0.92,...,60.95,0.205735,0.084164,4055040,3.87,4411392,3.05,2965504,3.50,2895952
1248,38,4269079,1241096,797144,1196768,1445888,1468006400,1.37,249120,0.91,...,78.14,0.210007,0.096677,4067328,3.88,4472832,3.09,3026944,3.64,2950168


In [36]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [37]:
numerical_result

,cluster0,cluster1
save0_0,0.015733,0.020889
save0_1,0.023035,0.035472
save1_0,0.021269,0.011167
save1_1,0.020059,0.021174
save2_0,0.018761,0.012396
save2_1,0.032575,0.024612
maxmemory,0.130318,0.029468
maxmemory-samples,0.019016,0.029924
hash-max-ziplist-entries,0.028476,0.017354
hash-max-ziplist-value,0.025595,0.050096


In [38]:
get_cluster(numerical_result)

save0_0 : cluster1
save0_1 : cluster1
save1_0 : cluster0
save1_1 : cluster1
save2_0 : cluster0
save2_1 : cluster0
maxmemory : cluster0
maxmemory-samples : cluster1
hash-max-ziplist-entries : cluster0
hash-max-ziplist-value : cluster1
hz : cluster0


In [52]:
numerical_result

,cluster0,cluster1
save0_0,0.006890,0.020577
save0_1,0.010659,0.018156
save1_0,0.014246,0.022073
save1_1,0.015685,0.012419
save2_0,0.014089,0.012587
save2_1,0.019130,0.014889
maxmemory,0.137578,0.007652
maxmemory-samples,0.016479,0.005751
hash-max-ziplist-entries,0.010083,0.014841
hash-max-ziplist-value,0.014439,0.013643


In [53]:
# extract_pvalue_knob_outlier(numerical_var, subTotal_knobs_T,total_internal_T)

---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [39]:
numericalActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [40]:
extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

###save0_0###
pearson
                        corr   p-value
lru_clock           0.094012  0.019312
used_memory_peak    0.103162  0.010220
rdb_last_save_time  0.094012  0.019312

kendalltau
                        corr   p-value
lru_clock           0.062364  0.020436
used_memory_peak    0.077819  0.017519
rdb_last_save_time  0.062364  0.020436

spearman
                        corr   p-value
lru_clock           0.093162  0.020438
used_memory_peak    0.095604  0.017349
rdb_last_save_time  0.093162  0.020438

###save0_1###
pearson
        corr   p-value
hz -0.096756  0.016038

kendalltau
        corr   p-value
hz -0.068438  0.016919

spearman
        corr   p-value
hz -0.094851  0.018254

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_1###
pearson
                               corr   p-value
hz                         0.088151  0

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [41]:
numericalNoActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [42]:
extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
                           corr   p-value
used_memory_rss        0.127823  0.001292
used_memory_rss_human  0.118833  0.002792

kendalltau
                             corr   p-value
allocator_allocated      0.067217  0.028602
used_memory_rss          0.090779  0.002381
used_memory_rss_human    0.092736  0.003871
mem_fragmentation_ratio  0.069905  0.023330
mem_fragmentation_bytes  0.075151  0.010292

spearman
                             corr   p-value
allocator_allocated      0.088073  0.026948
used_memory_rss          0.119924  0.002550
used_memory_rss_human    0.114887  0.003855
mem_fragmentation_ratio  0.090563  0.022902
mem_fragmentation_bytes  0.102347  0.010094

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Co

---

## pointbiserialr 분석

In [43]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

    
# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


# 이상치 제거 버전
def cal_pointbiserialr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pointbiserialr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue_outlier(knobs, internal_df):
    print('pointbierialr')
    cal_pointbiserialr(knobs, internal_df)
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [44]:
boolean_var

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'activerehashing',
 'dynamic-hz']

In [45]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [46]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                        corr   p-value
used_memory_peak    0.064518  0.022540
rss_overhead_bytes  0.062702  0.026635
used_cpu_user      -0.060374  0.032814
allocator_resident -0.056492  0.045838

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
                           corr   p-value
lru_clock             -0.090045  0.001438
rdb_last_save_time    -0.090044  0.001438
used_memory_rss        0.068435  0.015523
used_memory_rss_human  0.085152  0.002587

###activerehashing###
pointbierialr
                         corr   p-value
rss_overhead_bytes   0.067647  0.016755
allocator_resident  -0.067

In [47]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [48]:
bool_result

,cluster0,cluster1
rdbcompression,0.026417,0.031442
rdbchecksum,0.017687,0.014546
rdb-save-incremental-fsync,0.020104,0.013474
lazyfree-lazy-eviction,0.027099,0.012747
lazyfree-lazy-expire,0.025628,0.033862
lazyfree-lazy-server-del,0.027359,0.030931
activerehashing,0.030674,0.041208
dynamic-hz,0.029219,0.029313


In [49]:
get_cluster(bool_result)

rdbcompression : cluster1
rdbchecksum : cluster0
rdb-save-incremental-fsync : cluster0
lazyfree-lazy-eviction : cluster0
lazyfree-lazy-expire : cluster1
lazyfree-lazy-server-del : cluster1
activerehashing : cluster1
dynamic-hz : cluster1


In [57]:
#이상치 제거
extract_pvalue_knob_point_outlier(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

---

In [ ]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)